In [1]:
import tensorflow as tf
import numpy as np

In [2]:
'''
Tensorflow optimizers:
class tf.train.GradientDescentOptimizer   #for beginner : SGD is slowest
class tf.train.AdadeltaOptimizer
class tf.train.AdagradOptimizer
class tf.train.AdagradDAOptimizer
class tf.train.MomentumOptimizer   #adds momentum and updates learning rate
class tf.train.AdamOptimizer
class tf.train.FtrlOptimizer
class tf.train.RMSPropOptimizer #for games like Atari they use RMS Optimizer
'''

'\nTensorflow optimizers:\nclass tf.train.GradientDescentOptimizer   #for beginner : SGD is slowest\nclass tf.train.AdadeltaOptimizer\nclass tf.train.AdagradOptimizer\nclass tf.train.AdagradDAOptimizer\nclass tf.train.MomentumOptimizer   #adds momentum and updates learning rate\nclass tf.train.AdamOptimizer\nclass tf.train.FtrlOptimizer\nclass tf.train.RMSPropOptimizer #for games like Atari they use RMS Optimizer\n'

In [3]:
#tensorboard tutorials below

In [4]:
import matplotlib
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

#defining the structure of a layer

def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    with tf.name_scope('layer'):
        layer_name = "layer_%s" % n_layer
        #inputs from previous layer
        #in_size: size(number of neurons) of previous layer
        #out_size: size(number of neurons) of this layer
        #activation_function: used in this layer
        with tf.name_scope('Weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size]),name='W')
            tf.histogram_summary(layer_name+"/Weights",Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.histogram_summary(layer_name+"/biases",biases)
        with tf.name_scope('logit'):
            logit = tf.matmul(inputs,Weights) + biases
            tf.histogram_summary(layer_name+"/logit",logit)
        if activation_function is None:
            output = logit
        else:
            output = activation_function(logit,)
        tf.histogram_summary(layer_name+"/Output",output)
        return output

#make up some real data
x = np.linspace(-1,1,300)[:,np.newaxis]
noise = np.random.normal(0,0.05,x.shape)
y = np.square(x)-0.05+noise

#define placeholder for inputs to network
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,1],name='x_input')  #numerous rows and 1 column : structure of x
    ys = tf.placeholder(tf.float32,[None,1],name='y_input')  #numerous rows and 1 column : structure of y

#add hidden layer
layer1 = add_layer(xs,1,10,1,tf.nn.relu)#input size has one feature(unit)&the next hidden layer consists of 10 units

#add output layer
predicted = add_layer(layer1,10,1,2)#layer one had 10 units, output ys has 1 unit and since non linear regression
                                  #therefore, no activation function (try and put some activation function and 
                                  #check the results)
        
#error between prediction and real data
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(predicted-ys),reduction_indices=[1]))
    tf.scalar_summary("loss",loss)
with tf.name_scope('train'):
    training = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  #0.1 is the learning rate, minimise the loss

#important
init = tf.initialize_all_variables()
sess = tf.Session()

merged = tf.merge_all_summaries()

writer = tf.train.SummaryWriter("logs/",sess.graph)

sess.run(init)


for step in range(1000):
    sess.run(training,feed_dict={xs:x,ys:y})
    if(step%50==0):
        result = sess.run(merged,feed_dict={xs:x,ys:y})
        writer.add_summary(result,step)

In [6]:
#in order to visualise the tensorboard
#open the terminal and go to the directory containing the above logs folder/directory
#$ tensorboard --logdir=="logs"